<a href="https://colab.research.google.com/github/aegonwolf/ukraine-conflict-social-news/blob/main/Playing_with_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# DS toolkit
import pandas as pd

# Visualisation
import folium
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

Download and unzip the dataset using this cell. Make sure to upload the API token (kaggle.json) before running it.

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows'
!unzip /content/ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Let's explore the data!

In [ ]:
df = pd.read_csv(
    "UkraineCombinedTweetsDeduped_MAR10.csv.gzip", 
    compression="gzip",
    index_col="Unnamed: 0"
)

In [55]:
reduceddf = df.sample(10 ** 2, random_state = 42)
reduceddf.tail()

,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,retweetcount,text,hashtags,language,coordinates,favorite_count,extractedts
4808434,2462530824,idntwannabeu_,♡ @_chowoodz🧡💙✦ @treasuremembers 💎✨ • YG STAN ...,แอคหลุมหวีดพิยอน • อนิเมะ💥🐾,391,7,1035,2014-04-25 02:27:51.000000,1501910519715799041,2022-03-10 13:19:06,92204,เด็กชายตัวน้อยๆคนนี้ เดินร้องไห้ หนีสงครามจากย...,[],th,NaN,0,2022-03-10 13:21:47.637511
4912399,1261199216157876225,MichaelPitz1,Reisebusunternehmen,"Offenbach am Main, Deutschland",88,31,650,2020-05-15 07:38:31.000000,1501988599075643395,2022-03-10 18:29:22,676,Wir dürfen nicht schon wieder naiv werden. #Pu...,"[{'text': 'Putin', 'indices': [59, 65]}]",de,NaN,0,2022-03-10 18:36:47.595256
4866540,820344569527226370,upchuck66,"Democracy isn’t just on the ballot this time, ...",NaN,2517,1834,463294,2017-01-14 18:59:21.000000,1501954152624275456,2022-03-10 16:12:29,91,"In 2014, I addressed the brave #Ukrainian peop...","[{'text': 'Ukrainian', 'indices': [40, 50]}, {...",en,NaN,0,2022-03-10 16:21:34.843277
4643612,1110814508992344064,TeemuTukimies,NaN,NaN,580,476,44108,2019-03-27 08:03:11.000000,1501781384792264708,2022-03-10 04:45:58,1,@veronikahonka Ai niinkuin entinen tyttö oli v...,"[{'text': 'NATO', 'indices': [114, 119]}]",fi,NaN,0,2022-03-10 04:54:17.367254
4795783,717092348,AndreaTiberio7,Senza conflitto non si progredisce. Senza conf...,NaN,1905,2515,8208,2012-07-26 00:57:10.000000,1501900151903899653,2022-03-10 12:37:54,0,Ma tutti quelli che criticano #Zelensky per no...,"[{'text': 'Zelensky', 'indices': [30, 39]}, {'...",it,NaN,1,2022-03-10 12:39:52.701083


In [41]:
groupByLocdf = reduceddf.groupby(['location'])['tweetid'].count() # SELECT location, COUNT(tweetid) FROM reduceddf GROUP BY location

location
404 location not found    1
47.973906, 38.763963      1
Aeroship Iditarod         1
Anjou, Montréal           1
Ballantrae, Scotland      1
Name: tweetid, dtype: int64

In [56]:
def put_markers(map, df):
    geo_locator = Nominatim()
    for index, row in df.iterrows():
        location = row['location']
        if location:
            try:
                location = geo_locator.geocode(location)
            except GeocoderTimedOut:
                continue
            if location:
                folium.Marker([location.latitude, location.longitude], popup = row['location']).add_to(map)

In [58]:
map = folium.Map(location=[0, 0], zoom_start=2)
#groupByLocdf = groupByLocdf.reset_index()
put_markers(map, groupByLocdf)
map.save("index.html")

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning
